In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ccal0507/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
data = pd.read_csv('lyrics.csv')
data.head()

,artist,title,lyrics,description,release date
0,Ray Charles,Hit the Road Jack,"Hit the Road Jack Lyrics\nHit the road Jack, a...",This tongue and cheek verbal duel of a couple ...,August 1961
1,Ray Charles,Georgia On My Mind,Georgia On My Mind Lyrics\nGeorgia\nGeorgia\nT...,Written by Hoagy Carmichael and Stuart Gorrell...,September 1960
2,Ray Charles,I’ve Got a Woman,I’ve Got a Woman Lyrics\nWell\n\nI got a woman...,Ray Charles released “I’ve Got a Woman” as a s...,December 1954
3,Ray Charles,I Can’t Stop Loving You,I Can’t Stop Loving You Lyrics\n(I can't stop ...,?,1962
4,Ken Nordine,Yellow,"Yellow LyricsIn the beginning--\nOh, long befo...",?,"January 1, 1966"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist        385 non-null    object
 1   title         385 non-null    object
 2   lyrics        382 non-null    object
 3   description   385 non-null    object
 4   release date  376 non-null    object
dtypes: object(5)
memory usage: 15.2+ KB


In [5]:
df = data.query("description != '?'")
df.head()

,artist,title,lyrics,description,release date
0,Ray Charles,Hit the Road Jack,"Hit the Road Jack Lyrics\nHit the road Jack, a...",This tongue and cheek verbal duel of a couple ...,August 1961
1,Ray Charles,Georgia On My Mind,Georgia On My Mind Lyrics\nGeorgia\nGeorgia\nT...,Written by Hoagy Carmichael and Stuart Gorrell...,September 1960
2,Ray Charles,I’ve Got a Woman,I’ve Got a Woman Lyrics\nWell\n\nI got a woman...,Ray Charles released “I’ve Got a Woman” as a s...,December 1954
8,Ghetto Brothers,Girl From The Mountain,Girl From The Mountain Lyrics\nNa-na-na na-na-...,The only track on Power-Fuerza not written by ...,"January 1, 1971"
11,Ghetto Brothers,Viva Puerto Rico Libre,Viva Puerto Rico Libre LyricsBuenas\nAmados he...,Benji Melendez’s family moved from Puerto Rico...,"January 1, 1972"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 384
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist        318 non-null    object
 1   title         318 non-null    object
 2   lyrics        316 non-null    object
 3   description   318 non-null    object
 4   release date  315 non-null    object
dtypes: object(5)
memory usage: 14.9+ KB


# Data Cleaning

In [7]:
df['lyrics'][:2].values

array(["Hit the Road Jack Lyrics\nHit the road Jack, and don'cha come back\nNo more, no more, no more, no more\nHit the road Jack, and don'cha come back no more\nWhat'd you say?\nHit the road Jack, and don'cha come back\nNo more, no more, no more, no more\nHit the road Jack, and don'cha come back no more\nOld woman old woman, oh you treat me so mean\nYou're the meanest old woman that I've ever seen\nI guess if you say so\nI'll have to pack my things and go (that's right)\nHit the road Jack, and don'cha come back\nNo more, no more, no more, no more\nHit the road Jack, and don'cha come back no more\nWhat'd you say?\nHit the road Jack, and don'cha come back\nNo more, no more, no more, no more\nHit the road Jack, and don'cha come back no more\nNow baby, listen baby, don't you treat me this way\nCause I'll be back on my feet some day\nDon't care if you do, cause it's understood\nYou ain't got no money, you just a no good\nWell I guess if you say so\nI'll have to pack my things and go (that'

In [9]:
test_str = df['lyrics'][0].lower()

In [10]:
test_str

"hit the road jack lyrics\nhit the road jack, and don'cha come back\nno more, no more, no more, no more\nhit the road jack, and don'cha come back no more\nwhat'd you say?\nhit the road jack, and don'cha come back\nno more, no more, no more, no more\nhit the road jack, and don'cha come back no more\nold woman old woman, oh you treat me so mean\nyou're the meanest old woman that i've ever seen\ni guess if you say so\ni'll have to pack my things and go (that's right)\nhit the road jack, and don'cha come back\nno more, no more, no more, no more\nhit the road jack, and don'cha come back no more\nwhat'd you say?\nhit the road jack, and don'cha come back\nno more, no more, no more, no more\nhit the road jack, and don'cha come back no more\nnow baby, listen baby, don't you treat me this way\ncause i'll be back on my feet some day\ndon't care if you do, cause it's understood\nyou ain't got no money, you just a no good\nwell i guess if you say so\ni'll have to pack my things and go (that's right

In [12]:
from nltk.tokenize import word_tokenize

In [25]:
test_str_tokenized = np.array(word_tokenize(test_str))
test_str_tokenized

array(['hit', 'the', 'road', 'jack', 'lyrics', 'hit', 'the', 'road',
       'jack', ',', 'and', "don'cha", 'come', 'back', 'no', 'more', ',',
       'no', 'more', ',', 'no', 'more', ',', 'no', 'more', 'hit', 'the',
       'road', 'jack', ',', 'and', "don'cha", 'come', 'back', 'no',
       'more', 'what', "'d", 'you', 'say', '?', 'hit', 'the', 'road',
       'jack', ',', 'and', "don'cha", 'come', 'back', 'no', 'more', ',',
       'no', 'more', ',', 'no', 'more', ',', 'no', 'more', 'hit', 'the',
       'road', 'jack', ',', 'and', "don'cha", 'come', 'back', 'no',
       'more', 'old', 'woman', 'old', 'woman', ',', 'oh', 'you', 'treat',
       'me', 'so', 'mean', 'you', "'re", 'the', 'meanest', 'old', 'woman',
       'that', 'i', "'ve", 'ever', 'seen', 'i', 'guess', 'if', 'you',
       'say', 'so', 'i', "'ll", 'have', 'to', 'pack', 'my', 'things',
       'and', 'go', '(', 'that', "'s", 'right', ')', 'hit', 'the', 'road',
       'jack', ',', 'and', "don'cha", 'come', 'back', 'no', 'more', '

In [38]:
index = np.where(test_str_tokenized == 'lyrics')[0][0]
index

4

In [ ]:
embed

In [42]:
test_str_tokenized[index + 1:len(test_str_tokenized) - 1]

array(['hit', 'the', 'road', 'jack', ',', 'and', "don'cha", 'come',
       'back', 'no', 'more', ',', 'no', 'more', ',', 'no', 'more', ',',
       'no', 'more', 'hit', 'the', 'road', 'jack', ',', 'and', "don'cha",
       'come', 'back', 'no', 'more', 'what', "'d", 'you', 'say', '?',
       'hit', 'the', 'road', 'jack', ',', 'and', "don'cha", 'come',
       'back', 'no', 'more', ',', 'no', 'more', ',', 'no', 'more', ',',
       'no', 'more', 'hit', 'the', 'road', 'jack', ',', 'and', "don'cha",
       'come', 'back', 'no', 'more', 'old', 'woman', 'old', 'woman', ',',
       'oh', 'you', 'treat', 'me', 'so', 'mean', 'you', "'re", 'the',
       'meanest', 'old', 'woman', 'that', 'i', "'ve", 'ever', 'seen', 'i',
       'guess', 'if', 'you', 'say', 'so', 'i', "'ll", 'have', 'to',
       'pack', 'my', 'things', 'and', 'go', '(', 'that', "'s", 'right',
       ')', 'hit', 'the', 'road', 'jack', ',', 'and', "don'cha", 'come',
       'back', 'no', 'more', ',', 'no', 'more', ',', 'no', 'more', ','

In [ ]:
def clean_lyrics(llst, title):
    

In [ ]:
##### Cleaning lyrical data #####

# Step 1: Turn all letters into lowercase 

# Step 2: Remove {song title lyrics: } from each lyric 